In [1]:
import torch
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)
from tsai.all import *
import sklearn.metrics as skm
import sys
from sklearn.model_selection import train_test_split
import numpy as np
import random

# import outsiders
sys.path.append('..')
import data_preprocessing
my_setup()

/home1/eitan.c@staff.technion.ac.il/ECG_proj/dockvenv/code/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-eitan.c@staff.technion.ac.il'
2023-05-06 21:34:55.137261: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 21:35:04.426361: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-06 21:35:04.428

os              : Linux-5.15.0-69-generic-x86_64-with-glibc2.29
python          : 3.8.5
tsai            : 0.3.6
fastai          : 2.7.12
fastcore        : 1.5.29
torch           : 1.13.1+cu117
device          : 1 gpu (['NVIDIA GeForce RTX 3090'])
cpu cores       : 8
threads per cpu : 2
RAM             : 62.71 GB
GPU memory      : [24.0] GB


In [2]:
torch.cuda.is_available()

True

In [2]:
# prepare the data
def split_data_indexes(len_data, train_ratio):
    indexes = list(range(len_data))
    random.shuffle(indexes)
    train_size = int(len_data * train_ratio)
    train_indexes = indexes[:train_size]
    test_indexes = indexes[train_size:]
    return (train_indexes, test_indexes)
# ssh remote
swell_data = data_preprocessing.load_data(os.path.join("../swell_dataset", "swell_dict_index_6.npy")) 
swell_data = data_preprocessing.swell_prepare_for_10fold(swell_data)
swell_data[:, 1:4] -= 1
x = swell_data[:, 4:]

y_arousal = swell_data[:, 2]
y_arousal = y_arousal.reshape(y_arousal.shape[0],)

y_valence = swell_data[:, 3]
y_valence = y_valence.reshape(y_valence.shape[0],)

y_stress = swell_data[:, 1]
y_stress = y_stress.reshape(y_stress.shape[0],)

print(x.shape)
print(type(x))
print(y_stress.shape)
print(y_valence.shape)
print(y_arousal.shape)


# set X to 3-Dim array
x = np.array(x)
x = np.expand_dims(x, axis=1)
print(x.shape)
filename = 'X_memmap.npy'
X = np.memmap(filename, dtype='float32', mode='w+', shape=x.shape)
X[:] = x[:]
print(X.shape)
print(type(X))

filename = 'Y_arousal_memmap.npy'
Y_arousal = np.memmap(filename, dtype='float32', mode='w+', shape=y_arousal.shape)
Y_arousal[:] = y_arousal[:]

filename = 'Y_valence_memmap.npy'
Y_valence = np.memmap(filename, dtype='float32', mode='w+', shape=y_valence.shape)
Y_valence[:] = y_valence[:]

filename = 'Y_stress_memmap.npy'
Y_stress = np.memmap(filename, dtype='float32', mode='w+', shape=y_stress.shape)
Y_stress[:] = y_stress[:]
print(Y_stress.shape)
print(type(Y_stress))
splits=split_data_indexes(X.shape[0],0.9)
print(len(splits[0]))
print(len(splits[1]))

(14595, 2560)
<class 'numpy.ndarray'>
(14595,)
(14595,)
(14595,)
(14595, 1, 2560)
(14595, 1, 2560)
<class 'numpy.memmap'>
(14595,)
<class 'numpy.memmap'>
13135
1460


In [4]:
print(np.max(y_stress))
print(np.min(y_stress))
print(np.max(y_arousal))
print(np.min(y_arousal)) 
print(np.max(y_valence))
print(np.min(y_valence)) 
print(Y_stress)
print(Y_arousal)
print(Y_valence)
np.max(Y_stress)

2.0
0.0
8.0
0.0
8.0
1.0
[0. 0. 0. ... 0. 0. 0.]
[3. 3. 3. ... 1. 1. 1.]
[5. 5. 5. ... 6. 6. 6.]


2.0

**Prepare dataset loaders and model**

In [3]:
tfms  = [None, [Categorize()]]
splits=np.load('./splits_B.npy',allow_pickle=True)
splits=(splits[0],splits[1])
dsets_arousal = TSDatasets(X, Y_arousal, tfms=tfms, splits=splits, inplace=True)
dsets_valence = TSDatasets(X, Y_valence, tfms=tfms, splits=splits, inplace=True)
dsets_stress = TSDatasets(X, Y_stress, tfms=tfms, splits=splits, inplace=True)

bs=128
dls_stress = TSDataLoaders.from_dsets(dsets_stress.train, dsets_stress.valid, bs=bs, batch_tfms=[TSStandardize()], num_workers=0)
dls_arousal = TSDataLoaders.from_dsets(dsets_arousal.train, dsets_arousal.valid, bs=bs, batch_tfms=[TSStandardize()], num_workers=0)
dls_valence = TSDataLoaders.from_dsets(dsets_valence.train, dsets_valence.valid, bs=bs, batch_tfms=[TSStandardize()], num_workers=0)

model_stress = InceptionTime(dls_stress.vars, dls_stress.c)
model_arousal = InceptionTime(dls_arousal.vars, dls_arousal.c)
model_valence = InceptionTime(dls_valence.vars, dls_valence.c)

learn_stress = Learner(dls_stress, model_stress, metrics=accuracy)
learn_arousal = Learner(dls_arousal, model_arousal, metrics=accuracy)
learn_valence = Learner(dls_valence, model_valence, metrics=accuracy)

learn_stress.save('S0')
learn_arousal.save('S0')
learn_valence.save('S0')

Path('models/S0.pth')

In [5]:
print(model_stress)

InceptionTime(
  (inceptionblock): InceptionBlock(
    (inception): ModuleList(
      (0): InceptionModule(
        (convs): ModuleList(
          (0): Conv1d(1, 32, kernel_size=(39,), stride=(1,), padding=(19,), bias=False)
          (1): Conv1d(1, 32, kernel_size=(19,), stride=(1,), padding=(9,), bias=False)
          (2): Conv1d(1, 32, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
        )
        (maxconvpool): Sequential(
          (0): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
          (1): Conv1d(1, 32, kernel_size=(1,), stride=(1,), bias=False)
        )
        (concat): Concat(dim=1)
        (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU()
      )
      (1): InceptionModule(
        (bottleneck): Conv1d(128, 32, kernel_size=(1,), stride=(1,), bias=False)
        (convs): ModuleList(
          (0): Conv1d(32, 32, kernel_size=(39,), stride=(1,), padding=(19,), bias=False)
 

**Train for stress**

In [6]:
learn_stress.load('S0')
learn_stress.lr_find()

SuggestedLRs(valley=0.0020892962347716093)

In [4]:
learn_stress.fit_one_cycle(100,lr_max=1e-3)
learn_stress.save('S1')

epoch,train_loss,valid_loss,accuracy,time
0,1.432568,1.504460,0.257890,00:24
1,1.232076,1.461440,0.305227,00:09
2,1.112743,1.405773,0.310651,00:09
3,1.055276,1.436983,0.248028,00:09


In [8]:
learn_stress.save_all(path='./stress_inc', dls_fname='dls_stress', model_fname='model_stress', learner_fname='learner_stress')

**Train for arousal**

In [9]:
learn_arousal.load('S0')
learn_arousal.lr_find()

SuggestedLRs(valley=0.0003311311302240938)

In [ ]:
learn_arousal.fit_one_cycle(100,lr_max=1e-4)
learn_arousal.save('S1')

epoch,train_loss,valid_loss,accuracy,time
0,2.197161,2.114778,0.167123,00:09
1,2.074747,2.014642,0.234932,00:09
2,1.989311,1.928762,0.287671,00:09
3,1.910060,1.860339,0.332192,00:09
4,1.843829,1.806120,0.339041,00:09
5,1.801920,1.758067,0.344521,00:09
6,1.771609,1.716871,0.343151,00:09
7,1.742252,1.690797,0.346575,00:09
8,1.720527,1.675670,0.366438,00:09
9,1.683166,1.627590,0.348630,00:09


Path('models/S1.pth')

In [11]:
learn_arousal.save_all(path='./arousal_inc', dls_fname='dls_arousal', model_fname='model_arousal', learner_fname='learner_arousal')

**Train for valence**

In [12]:
learn_valence.load('S0')
learn_valence.lr_find()

SuggestedLRs(valley=0.001737800776027143)

In [ ]:
learn_valence.fit_one_cycle(100,lr_max=1e-3)
learn_valence.save('S1')

epoch,train_loss,valid_loss,accuracy,time
0,1.824867,1.669051,0.415753,00:09
1,1.667524,1.555377,0.445890,00:09
2,1.597406,1.515187,0.407534,00:09
3,1.559265,1.440574,0.456849,00:09
4,1.524295,1.403774,0.485616,00:09
5,1.500435,1.415810,0.449315,00:09
6,1.435993,1.328974,0.509589,00:09
7,1.339625,1.526630,0.421918,00:09
8,1.229593,1.294035,0.504795,00:09
9,1.099538,1.060807,0.623288,00:09


Path('models/S1.pth')

In [ ]:
print("end")

In [14]:
learn_valence.save_all(path='./valence_inc', dls_fname='dls_valence', model_fname='model_valence', learner_fname='learner_valence')

In [15]:
np.save('./splits_inc_2504',splits)

**Generaliztion**

In [4]:
def train_test_indexes_var2(ratio, objects):
    # Split objects into train and test sets
    num_test = int(len(set(objects)))-int(len(set(objects)) * ratio)
    obj_list = list(set(objects))
    random.shuffle(obj_list)
    test_objs = obj_list[:num_test]
    train_objs = obj_list[num_test:]

    train_idxs = []
    test_idxs = []

    # Assign each object's samples to the appropriate set
    for obj in train_objs:
        obj_idxs = [i for i, o in enumerate(objects) if o == obj]
        train_idxs.extend(obj_idxs)

    for obj in test_objs:
        obj_idxs = [i for i, o in enumerate(objects) if o == obj]
        test_idxs.extend(obj_idxs)

    # Shuffle the final train and test sets
    random.shuffle(train_idxs)
    random.shuffle(test_idxs)

    return (train_idxs), (test_idxs)

In [5]:
# check train_test_indexes_var2()
# not for each runnig acieve exaclty 9:1 train:test
ratio = 0.9
objects=swell_data[:,0]
print("tot len is ", len(objects))
print("trin len is ",ratio*len(objects))
print("test len is ",len(objects)-ratio*len(objects))
i=train_test_indexes_var2(ratio, objects)
#print("train idx ",i[0])
#print("test idx ",i[1])
print("train obj:")
tr=[]
for j in i[0]:
  #print(objects[int(j)])
  tr.append(objects[int(j)])
tr_u=set(tr)

print("test obj:")
te=[]
for j in i[1]:
  #print(objects[int(j)])
  te.append(objects[int(j)])
te_u=set(te)

print(tr_u,len(tr))
print(te_u,len(te))
splits=i

tot len is  14595
trin len is  13135.5
test len is  1459.5
train obj:
test obj:
{1.0, 3.0, 5.0, 7.0, 9.0, 10.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0} 12698
{2.0, 4.0, 6.0} 1897


In [6]:
tfms  = [None, [Categorize()]]

dsets_arousal = TSDatasets(X, Y_arousal, tfms=tfms, splits=splits, inplace=True)
dsets_valence = TSDatasets(X, Y_valence, tfms=tfms, splits=splits, inplace=True)
dsets_stress = TSDatasets(X, Y_stress, tfms=tfms, splits=splits, inplace=True)

bs=128
dls_stress = TSDataLoaders.from_dsets(dsets_stress.train, dsets_stress.valid, bs=bs, batch_tfms=[TSStandardize()], num_workers=0)
dls_arousal = TSDataLoaders.from_dsets(dsets_arousal.train, dsets_arousal.valid, bs=bs, batch_tfms=[TSStandardize()], num_workers=0)
dls_valence = TSDataLoaders.from_dsets(dsets_valence.train, dsets_valence.valid, bs=bs, batch_tfms=[TSStandardize()], num_workers=0)

model_stress = InceptionTime(dls_stress.vars, dls_stress.c)
model_arousal = InceptionTime(dls_arousal.vars, dls_arousal.c)
model_valence = InceptionTime(dls_valence.vars, dls_valence.c)

learn_stress = Learner(dls_stress, model_stress, metrics=accuracy)
learn_arousal = Learner(dls_arousal, model_arousal, metrics=accuracy)
learn_valence = Learner(dls_valence, model_valence, metrics=accuracy)

learn_stress.save('S0')
learn_arousal.save('S0')
learn_valence.save('S0')

Path('models/S0.pth')

In [7]:
learn_stress.load('S0')
learn_stress.lr_find()

SuggestedLRs(valley=0.0010000000474974513)

In [8]:
learn_stress.fit_one_cycle(100,lr_max=1e-3)
learn_stress.save('S1')

epoch,train_loss,valid_loss,accuracy,time
0,1.788600,1.996881,0.267264,00:09
1,1.438737,1.815022,0.289931,00:09
2,1.239650,1.659536,0.231945,00:09
3,1.114837,1.451738,0.366368,00:09
4,1.038356,1.569037,0.178703,00:09
5,0.996895,1.389922,0.232999,00:09
6,0.973628,1.416895,0.348445,00:09
7,0.954934,1.341174,0.230891,00:09
8,0.933404,1.515587,0.286241,00:09
9,0.914603,1.315361,0.331049,00:09


Path('models/S1.pth')